In [11]:
"""
参数回归-模型2

介绍：在模型1的基础上，优化波动函数。
考虑到币价波动往往是基于比例的，因此上涨时会越长越快，下跌时会越跌越慢。
假设币价不是均值函数和波动函数的叠加，而是基于两者相乘得到。

目标函数: Price(x) = M(x) * ( 1 + A(x) * S(x) )
M(x)为均值函数，A(x)为幅值函数，S(x)为周期函数。

参数:
0.1 c-(c>=0) y轴起点

1. M(x)=l*(x+x0)^k + c
参数：
1.1 L-(L>0)均值增长的速率
1.2 x0-(x0>0)增长曲线的起始
1.3 k-(0<k<1)控制增速衰减速度，值越小增速下降越快。

2. A(x)=a
参数：
2.1 a-(0<a<1)控制幅值

3. S(x)=sin(x/T + x1)
参数：
3.1 T-(T>0)周期
3.2 x2-(0<=x2<=2pi)初始相位
"""

'\n参数回归-模型2\n\n介绍：在模型1的基础上，优化波动函数。\n考虑到币价波动往往是基于比例的，因此上涨时会越长越快，下跌时会越跌越慢。\n假设币价不是均值函数和波动函数的叠加，而是基于两者相乘得到。\n\n目标函数: Price(x) = M(x) * ( 1 + A(x) * S(x) )\nM(x)为均值函数，A(x)为幅值函数，S(x)为周期函数。\n\n参数:\n0.1 c-(c>=0) y轴起点\n\n1. M(x)=l*(x+x0)^k + c\n参数：\n1.1 L-(L>0)均值增长的速率\n1.2 x0-(x0>0)增长曲线的起始\n1.3 k-(0<k<1)控制增速衰减速度，值越小增速下降越快。\n\n2. A(x)=a\n参数：\n2.1 a-(0<a<1)控制幅值\n\n3. S(x)=sin(x/T + x1)\n参数：\n3.1 T-(T>0)周期\n3.2 x2-(0<=x2<=2pi)初始相位\n'

In [32]:
# 导入包和数据
import math

import pandas as pd
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from lmfit import Model

# -------------------------------
# 1. 加载数据
# 假设CSV文件 'btc_prices.csv' 中包含 Date 和 Close 两列
df = pd.read_csv('btc_prices.csv', parse_dates=['open_date'], index_col='open_date')
df = df.sort_index()  # 确保按日期排序

# 将日期转换为数值（天数）用于回归分析
df['Time'] = np.arange(len(df))

x_data = df['Time']
y_data = df['open_price']
y_data_log = np.log(df['open_price'])

In [33]:
# 模型和参数
def fn_m(x, l:float, k:float, x0:float, c:float):
    return l*((x+x0)**k) + c

def fn_a(x, a:float):
    return a

def fn_s(x, x2: float, t:float):
    return np.sin(x/t + x2)

def fn_price(x, a:float, c:float, l:float, k:float, x0:float, x2:float, t:float):
    return fn_m(x, l, k, x0, c) * ( 1 + fn_a(x, a) * fn_s(x, x2, t))

def fn_log_price(x, a:float, c:float, l:float, k:float, x0:float, x2:float, t:float):
    prices = fn_price(x, a, c, l, k, x0, x2, t)
    # 将小于0的值都置为一个非常小的正数
    prices[prices < 0] = 1e-9
    return np.log(prices)

# 手动提供的参数
INIT_A = 0.8
INIT_C = 1
INIT_L = 600
INIT_K = 0.58
INIT_X0 = 1
# INIT_X1 = 600
INIT_X2 = math.pi * 0.7
INIT_T = 1400/math.pi/2

# # 回归获得的参数
# INIT_A = 5.68868564
# INIT_C = 291.096217
# INIT_L = 18.5985387
# INIT_K = 0.99999000
# INIT_X0 = 11.8565495
# INIT_X1 = 6.24777991
# INIT_X2 = 1.21494460
# INIT_T = 208.538659


# 创建 lmfit 模型
model = Model(fn_price)
model_log = Model(fn_log_price)

# 根据先验知识给出初始猜测，若无先验则可均设为1
# params = model.make_params(a=INIT_A, c=INIT_C, l=INIT_L, k=INIT_K, x0=INIT_X0, x2=INIT_X2, t=INIT_T)
params = model_log.make_params(a=INIT_A, c=INIT_C, l=INIT_L, k=INIT_K, x0=INIT_X0, x2=INIT_X2, t=INIT_T)

# 如果有参数边界、固定值或者其他约束，可使用：
params['a'].set(min=0+1e-9, max=1-1e-9) # a>0
params['c'].set(min=0+1e-9) # c>0
params['l'].set(min=0+1e-9) # L>0
params['k'].set(min=0+1e-9, max=1-1e-9) # 0<k<1
params['x0'].set(min=0+1e-9) # x0>0
params['x2'].set(min=0, max=2*math.pi) # 0<=x2<=2pi
params['t'].set(min=0) # T>0


# 准备数据
true_params = [INIT_A, INIT_C, INIT_L, INIT_K, INIT_X0, INIT_X2, INIT_T]
y_true = fn_price(x_data, *true_params) # 初始参数曲线
y_true_log = fn_log_price(x_data, *true_params) # 初始参数log曲线
y_true_m = fn_m(x_data, l=INIT_L, k=INIT_K, x0=INIT_X0, c=INIT_C) # 均值曲线

In [34]:
# 拟合数据
result = model_log.fit(y_data_log, params, x=x_data, method='trust-constr')
y_regression_fit = np.exp(result.best_fit)

# 输出拟合报告
print(result.fit_report())

# 最终参数
params = result.best_values

#
y_regression_fit_m = fn_m(x_data, l=params['l'], k=params['k'], x0=params['x0'], c=params['c'])
y_regression_fit_m_log = np.log(y_regression_fit_m)

[[Model]]
    Model(fn_log_price)
[[Fit Statistics]]
    # fitting method   = equality_constrained_sqp
    # function evals   = 1528
    # data points      = 2743
    # variables        = 7
    chi-square         = 395.075325
    reduced chi-square = 0.14439888
    Akaike info crit   = -5301.19626
    Bayesian info crit = -5259.77861
    R-squared          = 0.83058995
##  Warning: uncertainties could not be estimated:
    this fitting method does not natively calculate uncertainties
    and numdifftools is not installed for lmfit to do this. Use
    `pip install numdifftools` for lmfit to estimate uncertainties
    with this fitting method.
[[Variables]]
    a:   0.51902147 (init = 0.8)
    c:   57.0449914 (init = 1)
    l:   16.7856914 (init = 600)
    k:   1.00000000 (init = 0.58)
    x0:  147.287575 (init = 1)
    x2:  1.25298004 (init = 2.199115)
    t:   207.527652 (init = 222.8169)


In [21]:
# 创建log图表
fig = make_subplots()

# 添加log价格曲线
fig.add_trace(go.Scatter(x=df.index, y=y_data_log, mode='lines', name='比特币log价格'))

# 添加初始参数拟合曲线
fig.add_trace(go.Scatter(x=df.index, y=y_true_log, mode='lines', name='初始参数log拟合曲线'))

# 添加初始参数均值曲线
fig.add_trace(go.Scatter(x=df.index, y=np.log(y_true_m), mode='lines', name='初始参数log均值曲线'))

# 添加拟合价格曲线
fig.add_trace(go.Scatter(x=df.index, y=result.best_fit, mode='lines', name='拟合log价格'))

# 添加拟合log均值曲线
fig.add_trace(go.Scatter(x=df.index, y=y_regression_fit_m_log, mode='lines', name='拟合log均值曲线'))

# 更新布局
fig.update_layout(
    title='比特币拟合log价格',
    xaxis_title='日期',
    yaxis_title='价格',
    legend=dict(x=0, y=1)
)

# 显示图表
fig.show()

In [22]:
# 创建价格图表
fig = make_subplots()

# 添加实际价格曲线
fig.add_trace(go.Scatter(x=df.index, y=y_data, mode='lines', name='比特币价格'))

# 添加初始参数拟合曲线
fig.add_trace(go.Scatter(x=df.index, y=y_true, mode='lines', name='初始参数拟合曲线'))

# 添加初始参数均值曲线
fig.add_trace(go.Scatter(x=df.index, y=y_true_m, mode='lines', name='初始参数均值曲线'))

# 添加拟合价格曲线
fig.add_trace(go.Scatter(x=df.index, y=y_regression_fit, mode='lines', name='拟合价格'))

# 添加拟合log均值曲线
fig.add_trace(go.Scatter(x=df.index, y=y_regression_fit_m, mode='lines', name='拟合均值曲线'))

# 更新布局
fig.update_layout(
    title='比特币拟合价格',
    xaxis_title='日期',
    yaxis_title='价格',
    legend=dict(x=0, y=1)
)

# 显示图表
fig.show()